In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize
from analyze import *
import dunestyle.matplotlib as dunestyle
from read_waveform import data_reader

plt.style.use('dune.mplstyle')

In [ ]:
waveforms = np.array(
    data_reader(
        "data/20240115_SPE_LED365nm/SPE_365nm/run16_C1_LED_20ns_3V30/0_wave0_C1_LED_20ns_3V30.dat",
        10000,
        "all",
    )
)

In [ ]:
# calculate_waveform_parameters(waveforms)
pulse_start = 4100
pulse_end = 6000

# Waveforms

First, we display average waveform

In [ ]:
def plot_min(xs):
    range = xs.max() - xs.min()
    return xs.min() - range*0.05
def plot_max(xs):
    range = xs.max() - xs.min()
    return xs.max() + range*0.2
def set_ylim(xs):
    plt.ylim(plot_min(xs), plot_max(xs))

In [ ]:
average_waveform = np.mean(waveforms, axis=0)
plt.plot(average_waveform)
plt.suptitle("Average Acquisition")
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
set_ylim(average_waveform)
dunestyle.Preliminary()
plt.show()

as well as some typical waveforms,

## Acquisition Viewer

In [ ]:
j = 0
good_indices = [36, 31, 40, 41, 42]

In [ ]:
i = good_indices[j]
plt.plot(np.transpose(waveforms[i]))
plt.suptitle("Acquisition " + str(i))
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
set_ylim(waveforms[i])
dunestyle.Preliminary()
plt.show()
j += 1

# Baseline

Then, look at baseline

In [ ]:
plt.hist(
    waveforms[:, :pulse_start].flatten(),
    bins=100,
    log=True,
    histtype="step",
    #range=(2200, 2800)
)
plt.xlabel("Baseline noise [ADC]")
plt.ylabel("Samples")
dunestyle.Preliminary()
plt.show()

## Baseline Averages

In [ ]:
averages = np.mean(waveforms[:, :pulse_start], axis=1)
plt.hist(
    averages,
    bins=150,
    log=True,
    histtype="step",
    range=(2200, 2800)
    #range=(2300, 2380)
)
plt.xlabel("Baseline average [ADC]")
plt.ylabel("Acquisitions")
plt.show()

## Baseline Standard Deviation

In [ ]:
baseline_stds = np.std(waveforms[:, :pulse_start], axis=1)
plt.hist(
    baseline_stds,
    bins=150,
    log=True,
    histtype="step",
    range=(3, 15)
)
plt.xlabel("Baseline standard deviation [ADC]")
plt.ylabel("Acquisitions")
plt.axvline(5.5, color="#E69F00", linestyle="-.", label="std = 5.5")
plt.axvline(4.5, color="#D55E00", linestyle=":", label="std = 4.5")
plt.axvline(3.75, color="#56B4E9", linestyle="--", label="std = 3.75")
plt.legend()
plt.show()

I don't trust these peaks. I think it's likely that the second peak and after are cosmic PE peaks. Identify peak canditates.

## Comparison 2D Histogram

In [ ]:
plt.hist2d(averages, baseline_stds, range=((2200, 2800), (3, 400)), bins=(100, 100), norm="log")
plt.xlabel("Baseline average [ADC]")
plt.ylabel("Baseline std dev [ADC]")
plt.colorbar()
dunestyle.Preliminary()
plt.show()

Then, zooming in,

In [ ]:
plt.hist2d(averages, baseline_stds, range=((2300, 2380), (3, 15)), bins=(100, 100), norm="log")
plt.xlabel("Baseline average [ADC]")
plt.ylabel("Baseline std dev [ADC]")
plt.axhline(5.5, color="#E69F00", linestyle="-.", label="std = 5.5")
plt.axhline(4.5, color="#D55E00", linestyle=":", label="std = 4.5")
plt.axhline(3.75, color="#56B4E9", linestyle="--", label="std = 3.75")
plt.legend()
plt.colorbar()
dunestyle.Preliminary()
plt.show()

Let's see what making these cuts does to my average acquisition.

## Validating Cuts to Baseline

In [ ]:
plt.plot(np.mean(waveforms, axis=0), color="black", label="no cut")
plt.plot(np.mean(waveforms[baseline_stds < 5.5], axis=0), color="#E69F00", label="std < 5.5")
plt.plot(np.mean(waveforms[baseline_stds < 4.5], axis=0), color="#D55E00", label="std < 4.5")
plt.plot(np.mean(waveforms[baseline_stds < 3.75], axis=0), color="#56B4E9", label="std < 3.75")
plt.suptitle("Average Acquisition With Cut to Baseline Region")
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
plt.legend()
dunestyle.Preliminary()
plt.show()

I'm not sure which cut we should make. Let's see what some of these events look like in between the cuts.

## Acquisition Viewer

In [ ]:
test = waveforms[np.logical_and(3.75 <= baseline_stds, baseline_stds < 4.5)]
i = 9

In [ ]:
plt.plot(np.transpose(test[i,:pulse_start]))
plt.suptitle("Acquisition " + str(i))
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
set_ylim(test[i,:pulse_start])
dunestyle.Preliminary()
plt.show()
i += 1

Looking at about 20 of these acquisitions, it looks like all events whose baselines have a std between 3.75 and 4.5 contain cosmic peaks. This is good evidence to make the strictest cut, **std < 3.75**.

# Far Tail Region

This is the region from 8,000 to 10,000 samples.

Let's do the same thing for this region. But we can probably skip the average this time.

In [ ]:
tail_stds = np.std(waveforms[:, 8000:], axis=1)
plt.hist(
    tail_stds,
    bins=150,
    log=True,
    histtype="step",
    range=(3, 15)
)
plt.xlabel("Tail standard deviation [ADC]")
plt.ylabel("Acquisitions")
plt.axvline(6.1, color="#E69F00", linestyle="-.", label="std = 6.1")
plt.axvline(4.8, color="#D55E00", linestyle=":", label="std = 4.8")
plt.axvline(3.85, color="#56B4E9", linestyle="--", label="std = 3.85")
plt.legend()
dunestyle.Preliminary()
plt.show()

Nearly the same cuts seem appropriate here. Let's see what these do to my average waveform.

In [ ]:
plt.plot(np.mean(waveforms, axis=0), color="black", label="no cut")
plt.plot(np.mean(waveforms[tail_stds < 6.1], axis=0), color="#E69F00", label="std < 6.1")
plt.plot(np.mean(waveforms[tail_stds < 4.8], axis=0), color="#D55E00", label="std < 4.8")
plt.plot(np.mean(waveforms[tail_stds < 3.85], axis=0), color="#56B4E9", label="std < 3.85")
plt.suptitle("Average Acquisition With Cut to Tail Region")
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
plt.legend()
dunestyle.Preliminary()
plt.show()

## Acquisition Viewer

In [ ]:
#test = waveforms[np.logical_and(3.85 <= tail_stds, tail_stds < 4.8)]
test = waveforms[tail_stds < 3.85]
i = 0 #32

In [ ]:
plt.plot(np.transpose(test[i,pulse_start:]))
plt.suptitle("Acquisition " + str(i))
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
set_ylim(test[i,pulse_start:])
dunestyle.Preliminary()
plt.show()
i += 1

Looking at these events shows that I should do the strictest cut, **std < 3.85**.

# Near Tail Region

This is the region from 6,000 to 8,000 samples.

In [ ]:
decline_stds = np.std(waveforms[:, pulse_end:8000], axis=1)
plt.hist(
    decline_stds,
    bins=150,
    log=True,
    histtype="step",
    range=(3, 15)
)
plt.xlabel("Baseline standard deviation [ADC]")
plt.ylabel("Acquisitions")
plt.axvline(6.3, color="#E69F00", linestyle="-.", label="std = 6.3")
plt.axvline(4.9, color="#D55E00", linestyle=":", label="std = 4.9")
plt.axvline(3.9, color="#56B4E9", linestyle="--", label="std = 3.9")
plt.legend()
dunestyle.Preliminary()
plt.show()

Let's see what these cuts do to my average waveform

In [ ]:
plt.plot(np.mean(waveforms, axis=0), color="black", label="no cut")
plt.plot(np.mean(waveforms[decline_stds < 6.3], axis=0), color="#E69F00", label="std < 6.3")
plt.plot(np.mean(waveforms[decline_stds < 4.9], axis=0), color="#D55E00", label="std < 4.9")
plt.plot(np.mean(waveforms[decline_stds < 3.9], axis=0), color="#56B4E9", label="std < 3.9")
plt.suptitle("Average Acquisition With Cut to Decline Region")
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
plt.legend()
dunestyle.Preliminary()
plt.show()

## Acquisition Viewer

In [ ]:
test = waveforms[np.logical_and(4.9 <= tail_stds, tail_stds < 6.3)]
#test = waveforms[decline_stds < 3.85]
i = 6 #32

In [ ]:
plt.plot(np.transpose(test[i,pulse_start:]))
plt.suptitle("Acquisition " + str(i))
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
set_ylim(test[i,pulse_start:])
dunestyle.Preliminary()
plt.show()
i += 1

Looking at events such as the 6th one after the cut `np.logical_and(3.9 <= tail_stds, tail_stds < 4.9)`, 3.9 is likely too stringent a cut. I won't do a cut here unless I actually fit each peak, and cut based on $\chi^2$.

# Putting it All Together

In [ ]:
plt.plot(np.mean(waveforms, axis=0), color="black", label="no cut")
# plt.plot(np.mean(waveforms[(decline_stds < 3.9) & (tail_stds < 3.9) & (baseline_stds < 3.85)], axis=0), color="#56B4E9", label="cut")
# plt.plot(np.mean(waveforms[(decline_stds < 4.9) & (tail_stds < 3.9) & (baseline_stds < 3.85)], axis=0), color="#D55E00", label="cut")
plt.plot(np.mean(waveforms[(tail_stds < 3.9) & (baseline_stds < 3.85)], axis=0), color="gray", label="with cut")
plt.suptitle("Average Acquisition With Final Cut")
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
plt.legend()
dunestyle.Preliminary()
plt.show()

## Acquisition Viewer

Now, let's look at some acquisitions from this cut data.

In [ ]:
i = 0
acquisitions_cut = waveforms[(tail_stds < 3.9) & (baseline_stds < 3.85)]

In [ ]:
plt.plot(np.transpose(acquisitions_cut[i]))
plt.suptitle("Acquisition " + str(i))
plt.xlabel("Samples [2 ns]")
plt.ylabel("Signal [ADC]")
set_ylim(acquisitions_cut[i])
dunestyle.Preliminary()
plt.show()
i+= 1

## Finally, Multipeak Fit?

In [ ]:
acquisitions_cut = waveforms[(tail_stds < 4.8) & (baseline_stds < 4.5)]
baseline_subtracted = acquisitions_cut - np.mean(acquisitions_cut[:, :pulse_start], axis=1)[:, np.newaxis]
pulses = baseline_subtracted[:,pulse_start:pulse_end]
charges = np.sum(pulses, axis=1)

In [ ]:
plt.hist(
    charges,
    bins=200,
    log=True,
    histtype="step",
    range=(-2000, 30000)
)
plt.xlabel("Charge (Integral of pulse) [2 ADC ns]")
plt.ylabel("Acquisitions")
dunestyle.Preliminary()
plt.show()